In [26]:
import pandas as pd
import random

# Ratio of users to consider in warm start setting
include_unknown = False
warm_start_ratio = 0.75

ratings = pd.read_csv(f'mindreader/ratings-mr100k.csv')
if not include_unknown:
    ratings = ratings[ratings.sentiment != 0]

users = ratings['userId'].unique()
random.shuffle(users)

# Partition into warm and cold start users
num_warm_start = int(len(users) * warm_start_ratio)
warm_start_users = set(users[:num_warm_start])
cold_start_users = set(users[num_warm_start:])

assert warm_start_users.isdisjoint(cold_start_users)

# Partition ratings
warm_ratings = ratings[ratings.userId.isin(warm_start_users)]
cold_ratings = ratings[ratings.userId.isin(cold_start_users)]


        Unnamed: 0                                userId  \
0                0  d50f84c0-17cd-11ea-bd9f-33d41a12d743   
1                1  d50f84c0-17cd-11ea-bd9f-33d41a12d743   
2                2  d50f84c0-17cd-11ea-bd9f-33d41a12d743   
3                3  d50f84c0-17cd-11ea-bd9f-33d41a12d743   
4                4  d50f84c0-17cd-11ea-bd9f-33d41a12d743   
...            ...                                   ...   
102136      102136  b41d5180-18bc-11ea-9dab-c3faade56b4f   
102137      102137  b41d5180-18bc-11ea-9dab-c3faade56b4f   
102138      102138  b41d5180-18bc-11ea-9dab-c3faade56b4f   
102139      102139  b41d5180-18bc-11ea-9dab-c3faade56b4f   
102140      102140  b41d5180-18bc-11ea-9dab-c3faade56b4f   

                                            uri  isItem  sentiment  
0        http://www.wikidata.org/entity/Q637842    True          1  
1        http://www.wikidata.org/entity/Q221075    True          1  
2        http://www.wikidata.org/entity/Q471839   False          1  
3  